## << 문제 정의 >>

주어진 titanic dataset을 사용하여 탑승객별로 **생존 여부(Survived)**를 예측하는 문제를 풉니다.

사용할 데이터는 kaggle에서 제공하는 Titanic Data를 사용합니다. 데이터 설명은 아래 링크를 참고하세요.

평가하는 성능 지표는 예측한 Survived와 정답 Survived 사이의 **Accuracy**값으로 정의합니다.



> [Source] https://www.kaggle.com/competitions/titanic/data

In [1]:
# 0. 라이브러리 및 데이터 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
print(train.shape, test.shape)

(891, 12) (418, 11)


### Q1. train 데이터의 column별 결측치 비율과 결측치가 하나 이상 포함되어 있는 row의 수를 출력하세요.

In [2]:
# column별 결측치 비율 출력 코드
print("column별 결측치 비율: ")
missing_ratio = train.isnull().sum() / len(train) * 100
for col, ratio in missing_ratio.items():
    print(f"{col:<12}   :    {ratio:.2f}%")

print("결측치를 포함하고 있는 row 수 : ", train.isnull().any(axis=1).sum())

column별 결측치 비율: 
PassengerId    :    0.00%
Survived       :    0.00%
Pclass         :    0.00%
Name           :    0.00%
Sex            :    0.00%
Age            :    19.87%
SibSp          :    0.00%
Parch          :    0.00%
Ticket         :    0.00%
Fare           :    0.00%
Cabin          :    77.10%
Embarked       :    0.22%
결측치를 포함하고 있는 row 수 :  708


### Q2. train data에 결측치가 존재하는 column들을 다음과 같은 방법으로 처리하세요.

- Age column은 Age column의 중앙값으로 결측치를 채운다.

- Cabin column은 Cabin column 자체를 제거한다. (column drop)

- Embarked column은 "C"로 결측치를 채운다.

In [3]:
train.Age = train['Age'].fillna(train.Age.median())
train = train.drop(columns='Cabin')
train.Embarked = train.Embarked.fillna('C')

### Q3. 문자열을 포함하고 있는 column중에 의미가 있다고 판단되는 Sex, Embarked column에 One-Hot Encoding을 적용하세요.

In [4]:
train = pd.get_dummies(train, columns=['Sex', 'Embarked'])
train

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,False,True,False,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,True,False,True,False,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,True,False,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,True,False,False,False,True
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,False,True,False,False,True
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,True,False,False,False,True
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",28.0,1,2,W./C. 6607,23.4500,True,False,False,False,True
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,False,True,True,False,False


### Q4. 예측 대상인 Survived에 전혀 영향을 주지 않을 column들인 ["PassengerId", "Name", "Ticket"]를 제거하세요.

In [5]:
drop_cols = ['PassengerId', 'Name', 'Ticket']
train = train.drop(columns = drop_cols)
train

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,False,True,False,False,True
1,1,1,38.0,1,0,71.2833,True,False,True,False,False
2,1,3,26.0,0,0,7.9250,True,False,False,False,True
3,1,1,35.0,1,0,53.1000,True,False,False,False,True
4,0,3,35.0,0,0,8.0500,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,False,True,False,False,True
887,1,1,19.0,0,0,30.0000,True,False,False,False,True
888,0,3,28.0,1,2,23.4500,True,False,False,False,True
889,1,1,26.0,0,0,30.0000,False,True,True,False,False


### Q5. 1주차에 배웠던 코드를 사용해서, train 데이터를 3 : 1의 비율로 나눠서 train, validation 데이터를 만드세요.

In [6]:
# 4. train-validation split
from sklearn.model_selection import train_test_split

# feature vector
X = train.drop('Survived', axis = 1)
# target value
y = train.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(668, 10) (223, 10) (668,) (223,)


### Q6. Random Forest를 사용해서 모델을 학습하고, 학습 데이터와 검증 데이터에 대해 Accuracy를 계산하세요.
(이 때, hyper-parameter는 자유롭게 정해도 됩니다.)

In [7]:
# 예측에 사용할 모델을 가져와서 학습
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# 학습한 모델을 평가
from sklearn.metrics import accuracy_score

pred_train = clf.predict(X_train)
pred_val = clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, pred_train))
print("Validation ACC : %.4f" % accuracy_score(y_val, pred_val))

Train ACC : 0.9805
Validation ACC : 0.7848


### Q7. test 데이터를 앞에서 수행했던 모든 전처리를 그대로 이용해서 Q6에서 학습한 모델로 결과를 출력할 수 있게 만드세요.

학습한 모델로 출력 결과가 나온다는 것은 아래 작성된 `clf.predict(X_test)`를 실행했을 때 어떠한 warning이나 error없이 실행되는 것을 말합니다.

- 주의 : test data를 앞에서 사용한 방식으로 전처리를 하려고 하면, Fare column의 결측치를 처리할 방법이 없을겁니다. Fare column의 결측치는 train data의 Fare column의 평균값을 계산해서 채워주세요.

In [8]:
test['Age'] = test['Age'].fillna(train.Age.median())
test['Fare'] = test['Fare'].fillna(train.Fare.mean())
test = test.drop('Cabin', axis=1)
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])

drop_cols = ['PassengerId', 'Name', 'Ticket']
test = test.drop(columns = drop_cols)

X_test = test.reindex(columns=X_train.columns, fill_value=0)

# X_test를 만들어서 아래 코드를 실행하세요.
output = clf.predict(X_test)
assert len(output) == 418  # sanity check

### Q8. 계산한 output을 사용하여 제출 파일인 submission.csv를 만드세요. submission.csv의 Survived column에 output 결과를 덮어쓰면 됩니다.

제출 파일의 이름은 submission.csv이며 아래 코드의 submission DataFrame을 완성하면 자동으로 생성됩니다.

In [9]:
# submission DataFrame을 만들어서 아래 코드를 실행하세요.
submission = pd.read_csv("gender_submission.csv")
submission['Survived'] = output
submission.to_csv('submission.csv', index=False)